# FEgrow: An Open-Source Molecular Builder and Free Energy Preparation Workflow

**Authors: Mateusz K Bieniek, Ben Cree, Rachael Pirie, Joshua T. Horton, Natalie J. Tatum, Daniel J. Cole**

## Overview

Here we introduce the ChemSpace class which:
 - automates protocols and takes care of CPU/cluster processing with Dask
 - stores data in a dataframe
 - employs scikit for active learning


In [ ]:
import prody
from rdkit import Chem

import fegrow
from fegrow import RGroups, Linkers, ChemSpace

# initialise
rgroups = RGroups()
linkers = Linkers()

# Prepare the ligand template

The provided core structure `lig.pdb` has been extracted from a crystal structure of Mpro in complex with compound **4** from the Jorgensen study (PDB: 7L10), and a Cl atom has been removed to allow growth into the S3/S4 pocket. The template structure of the ligand is protonated with [Open Babel](http://openbabel.org/wiki/Main_Page):

In [ ]:
!obabel sarscov2/lig.pdb -O sarscov2/coreh.sdf -p 7

Load the protonated ligand into FEgrow:

In [ ]:
init_mol = Chem.SDMolSupplier('sarscov2/mini.sdf', removeHs=False)[0]

# get the FEgrow representation of the rdkit Mol
scaffold = fegrow.RMol(init_mol)

Show the 2D (with indices) representation of the core. This is used to select the desired growth vector.

In [ ]:
scaffold.rep2D(idx=True, size=(500, 500))

Using the 2D drawing, select an index for the growth vector. Note that it is currently only possible to grow from hydrogen atom positions. In this case, we are selecting the hydrogen atom labelled H:40 to enable growth into the S3/S4 pocket of Mpro.

In [ ]:
# specify the connecting point
scaffold.GetAtomWithIdx(8).SetAtomicNum(0)

In [ ]:
# create the chemical space
cs = ChemSpace()

In [ ]:
cs.add_scaffold(scaffold)

In [ ]:
# initially it is empty
cs

# Select RGroups for your template

R-groups can be selected interactively or programmaticaly.

We have provided a set of common R-groups (see `fegrow/data/rgroups/library`), which can be browsed and selected interactively below.

Molecules from the library can alternatively be selected by name, as demonstrated below.

Finally, user-defined R-groups may be provided as `.mol` files. *In this case, the hydrogen atom selected for attachment should be replaced by the element symbol R.* See the directory `manual_rgroups` for examples.

In [ ]:
rgroups

In [ ]:
# retrieve the interactively selected groups
interactive_rgroups = rgroups.get_selected()

# you can also directly access the built-in dataframe programmatically
R_group_ethanol = rgroups[rgroups.Name == '*CCO'].Mol.item()

# select the R-group using the index
R_group_cyclopropane = rgroups.Mol[69] 

# use SMILES
R_group_methanol = Chem.AddHs(Chem.MolFromSmiles('*CO'))

# add your R-groups from files
R_group_propanol = Chem.MolFromMolFile('manual_rgroups/propan-1-ol-r.mol', removeHs=False)

### Expand your chemical space by building on top off your scaffold. 

In [ ]:
# Adding R-groups implies that the scaffold should be used. 
# The previously scaffold will be attached automatically. 

# or we can use the template merged with the linker
# in which case the attachement point is not needed (R* atom is used)
cs.add_rgroups(R_group_ethanol)
cs

In [ ]:
linkers

In [ ]:
# get linkers programmatically from the library
rcr_linker = linkers[linkers.Name == 'R1CR2'].Mol.item()
rocr_linker = linkers.Mol[6], # use the linker table index directly, e.g. index 6 is "R2COR1"

# pick linkers from the grid
grid_linkers = linkers.get_selected()

# use Smiles
rcor_linker = Chem.AddHs(Chem.MolFromSmiles('[*:0]CO[*:1]'))

# 

### Add linkers to build more structures

In [ ]:
# Adding R-groups implies that the scaffold should be used. 
# The previously scaffold will be attached automatically. 

# join a linker with the rgroups
cs.add_rgroups(rcor_linker, [R_group_methanol, R_group_propanol])
cs

The R-group library can also be viewed as a 2D grid, or individual molecules can be selected for 3D view (note that the conformation of the R-group has not yet been optimised):

In [ ]:
mol = cs[0]
mol.rep2D()

In [ ]:
cs[0].rep3D()

Once the ligands have been generated, they can be assessed for various ADMET properties, including Lipinksi rule of 5 properties, the presence of unwanted substructures or problematic functional groups, and synthetic accessibility.

In [ ]:
cs.toxicity()

For each ligand, a specified number of conformers (`num_conf`) is generated by using the RDKit [ETKDG algorithm](https://doi.org/10.1021/acs.jcim.5b00654). Conformers that are too similar to an existing structure are discarded. Empirically, we have found that `num_conf=200` gives an exhaustive search, and `num_conf=50` gives a reasonable, fast search, in most cases.

If required, a third argument can be added `flexible=[0,1,...]`, which provides a list of additional atoms in the core that are allowed to be flexible. This is useful, for example, if growing from a methyl group and you would like the added R-group to freely rotate.

In [ ]:
cs.generate_conformers(num_conf=50, 
                          minimum_conf_rms=0.5, 
                          # flexible=[3, 18, 20])
                        )

### Prepare the protein

The protein-ligand complex structure is downloaded, and [PDBFixer](https://github.com/openmm/pdbfixer) is used to protonate the protein, and perform other simple repair:

In [ ]:
# get the protein-ligand complex structure
!wget -nc https://files.rcsb.org/download/7L10.pdb

# load the complex with the ligand
sys = prody.parsePDB('7L10.pdb')

# remove any unwanted molecules
rec = sys.select('not (nucleic or hetatm or water)')

# save the processed protein
prody.writePDB('rec.pdb', rec)

# fix the receptor file (missing residues, protonation, etc)
fegrow.fix_receptor("rec.pdb", "rec_final.pdb")

# load back into prody
rec_final = prody.parsePDB("rec_final.pdb")

View enumerated conformers in complex with protein:

In [ ]:
cs[0].rep3D(prody=rec_final)

Any conformers that clash with the protein (any atom-atom distance less than 1 Angstrom), are removed.

In [ ]:
cs.remove_clashing_confs(rec_final)

In [ ]:
cs[0].rep3D(prody=rec_final)

### Optimise conformers in context of protein

The remaining conformers are optimised using hybrid machine learning / molecular mechanics (ML/MM), using the [ANI2x](https://doi.org/10.1021/acs.jctc.0c00121) neural nework potential for the ligand energetics (as long as it contains only the atoms H, C, N, O, F, S, Cl). Note that the Open Force Field [Parsley](https://doi.org/10.1021/acs.jctc.1c00571) force field is used for intermolecular interactions with the receptor.

`sigma_scale_factor`: is used to scale the Lennard-Jones radii of the atoms.

`relative_permittivity`: is used to scale the electrostatic interactions with the protein.

`water_model`: can be used to set the force field for any water molecules present in the binding site.

In [ ]:
# opt_mol, energies
energies = cs.optimise_in_receptor(
    receptor_file="rec_final.pdb", 
    ligand_force_field="openff", 
    use_ani=True,
    sigma_scale_factor=0.8,
    relative_permittivity=4,
    water_model = None,
    platform_name = 'CPU', # or e.g. 'CUDA'
)

Any of the rmols that have no available conformers (due to unresolvable steric clashes with the protein) can be discarded using the `.discard_missing()` function. This function also returns a list of the indices that were removed, which can be helpful when carrying out data analysis.

In [ ]:
missing_ids = cs.discard_missing()

Optionally, display the final optimised conformers. Note that, unlike classical force fields, ANI allows bond breaking. You may occasionally see ligands with distorted structures and very long bonds, but in our experience these are rarely amongst the low energy structures and can be ignored.

Conformers are now sorted by energy, only retaining those within 5 kcal/mol of the lowest energy structure:

In [ ]:
final_energies = cs.sort_conformers(energy_range=5)

In [ ]:
cs[0].rep3D()

Save all of the lowest energy conformers to files and print the sorted energies in kcal/mol (shifted so that the lowest energy conformer is zero).

In [ ]:
cs.to_sdf("optimised_molecules.sdf")
cs[0].to_file("best_conformers0.pdb") 

In [ ]:
print(final_energies)

The conformers are scored using the [Gnina](https://github.com/gnina/gnina) molecular docking program and convolutional neural network scoring function. *[Note that this step is not supported on macOS].* If unavailable, the Gnina executable is downloaded during the first time it is used. The CNNscores may also be converted to predicted Kd (nM) (see column "Kd").

In [ ]:
affinities = cs.gnina(receptor_file="rec_final.pdb") 
affinities

Predicted binding affinities may be further refined using the structures output by `FEgrow`, using your favourite free energy calculation engine. See our paper for an example using [SOMD](https://github.com/michellab/Sire) to calculate the relative binding free energies of 13 Mpro inhibitors.

In [ ]:
# display units
affinities.Kd